<a href="https://colab.research.google.com/github/ShishirPandy/Lip_Sync/blob/main/Lip_Sync_using_DINET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Check gpu
!nvidia-smi

Sun Aug  6 18:38:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Download DINet
!git clone https://github.com/MRzzm/DINet
%cd /content/DINet

Cloning into 'DINet'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 43 (delta 8), reused 42 (delta 7), pack-reused 0
Receiving objects: 100% (43/43), 40.30 KiB | 13.43 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/DINet


In [3]:
#@title Download model and data
!gdown --id 1CKUlFdvVtH-f2BYlQbjqfkoRlC_7C0Ka
!unzip -o asserts.zip
!rm asserts.zip
!rm asserts.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1CKUlFdvVtH-f2BYlQbjqfkoRlC_7C0Ka
To: /content/DINet/asserts.zip
100% 1.01G/1.01G [00:14<00:00, 69.8MB/s]
Archive:  asserts.zip
   creating: asserts/
  inflating: asserts/clip_training_DINet_256mouth.pth  
   creating: asserts/examples/
  inflating: asserts/examples/driving_audio_1.wav  
  inflating: asserts/examples/driving_audio_2.wav  
  inflating: asserts/examples/driving_audio_3.wav  
  inflating: asserts/examples/driving_audio_4.wav  
  inflating: asserts/examples/driving_audio_5.wav  
  inflating: asserts/examples/driving_audio_6.wav  
  inflating: asserts/examples/test1.csv  
  inflating: asserts/examples/test1.mp4  
  inflating: asserts/examples/test10.csv  
  inflating: asserts/examples/test10.mp4  
  inflating: 

In [4]:
#@title Install Virtual Environment
# miniconda
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')
!conda install python=3.7 -y
!python --version
!conda create --name DINet python=3.7 -y
!source activate DINet

--2023-08-06 18:38:56--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   302MB/s    in 0.3s    

2023-08-06 18:38:56 (302 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=p

In [5]:
#@title Installation dependencies
!pip install  resampy==0.2.2  python_speech_features  opencv_python==3.4.18.65
!pip install  numpy===1.18.0  resampy==0.2.2      scipy==1.5.4
!pip install  torch==1.7.1   torchvision==0.8.2    protobuf==3.20.0
!pip install  tensorflow==1.15.2 -i https://pypi.tuna.tsinghua.edu.cn/simple

     |████████████████████████████████| 323 kB 23.8 MB/s 
     |████████████████████████████████| 58.4 MB 1.4 MB/s 
     |████████████████████████████████| 15.7 MB 58.4 MB/s 
     |████████████████████████████████| 38.1 MB 125 kB/s 
     |████████████████████████████████| 3.5 MB 65.6 MB/s 
     |████████████████████████████████| 34.6 MB 279 kB/s 
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320720 sha256=cb1834b2146a05c96731651fad08c74b3014e0e1b178326492965fd7dddf01ae
  Stored in directory: /root/.cache/pip/wheels/a0/18/0a/8ad18a597d8333a142c9789338a96a6208f1198d290ece356c
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=5b08d639d675d729d66dc5a16bb6abfc6b447f352da305272c5163e3e87792b4
  Stored in directory: /root/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built resampy python-speech-features
     |████████████████████████████████| 20.1 MB 1.2 MB/

In [6]:
#@title Official Inference
%cd /content/DINet
!python inference.py --mouth_region_size=256 --source_video_path=./asserts/examples/test4.mp4 --source_openface_landmark_path=./asserts/examples/test4.csv --driving_audio_path=./asserts/examples/driving_audio_4.wav --pretrained_clip_DINet_path=./asserts/clip_training_DINet_256mouth.pth
# The result in ./asserts/inference_result

/content/DINet
extracting frames from video: ./asserts/examples/test4.mp4
extracting deepspeech feature from : ./asserts/examples/driving_audio_4.wav
/content/DINet/utils/deep_speech.py:71: UserWarning: Audio has multiple channels, the first channel is used
  "Audio has multiple channels, the first channel is used")
2023-08-06 18:43:05.143398: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-08-06 18:43:05.149914: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000160000 Hz
2023-08-06 18:43:05.150093: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55a969469a00 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-08-06 18:43:05.150111: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-08-06 18:43:05.233382: W tensorflow/core/framework/cpu_all

In [7]:
#@title Custom Inference
# Use OpenFace_2.2.0_win_x64 to generate a CSV file for your own video.
%cd /content/DINet
!gdown https://drive.google.com/uc\?id\=1flufnx3tIZZLja2XsxPEBcwqI_IUZFNG
!unzip input.zip
!rm input.zip
!python inference.py --mouth_region_size=256 --source_video_path=./input/888.mp4 --source_openface_landmark_path=./input/888.csv --driving_audio_path=./input/888.wav --pretrained_clip_DINet_path=./asserts/clip_training_DINet_256mouth.pth
# The result in ./asserts/inference_result


/content/DINet
Downloading...
From: https://drive.google.com/uc?id=1flufnx3tIZZLja2XsxPEBcwqI_IUZFNG
To: /content/DINet/input.zip
100% 1.03M/1.03M [00:00<00:00, 125MB/s]
Archive:  input.zip
   creating: input/
  inflating: input/888.csv           
  inflating: input/888.mp4           
  inflating: input/888.wav           
extracting frames from video: ./input/888.mp4
extracting deepspeech feature from : ./input/888.wav
/content/DINet/utils/deep_speech.py:71: UserWarning: Audio has multiple channels, the first channel is used
  "Audio has multiple channels, the first channel is used")
2023-08-06 18:44:39.568383: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-08-06 18:44:39.574888: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000160000 Hz
2023-08-06 18:44:39.575055: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56a311679770 initial